<center><h1> Welcome To Dynamic SVB Report Demo</h1></center>

<center><h1>Yifeng Yuan</h1></center>

## Introduction ##
In this demo, you will see a few examples of implementing the following items:

1. Show all databases on the server
2. Show all tables in the database
3. Show total number of rows in the table
4. Enumerate all API versions in the table
5. Interactive pivot chart
    1. TX07 Single-tone Attenuation Sweep Test
        1. Pout vs Attenuation plot
        2. Tx Gain Delta vs Attenuation plot
        3. Tx LO Leakage vs Attenuation plot
    2. TX00 1-Tone Frequency Sweep Test
        1. Image Rejection Vs Baseband Tone Sweep Frequency
        2. TX Swept Tone Image Power Vs Baseband Tone Sweep Frequency
        3. TX Swept Tone Fundamental Power Vs Baseband Tone Sweep Frequency


In [1]:
# import all modules and libraries
import pyodbc
import xlwings as xw
import string
import pandas as pd
from pandas import DataFrame
from pivottablejs import pivot_ui
from IPython.display import HTML
# ------------------------------------
import ipypivot as pt
import matplotlib.pyplot as plt

from IPython.display import FileLink, FileLinks 

In [2]:
# connect to server
# create a cursor object
# def create_engine(server_name, db=''):
#     connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=dagger;DATABASE=test;Trusted_Connection"
# connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

def db_connection(server_name, db=''):

    str_connection = 'Driver={SQL Server};Server=' + str(server_name) + '; Trusted_Connection=yes; Database=' + str(db)
    conn = pyodbc.connect(str_connection)    
    cursor = conn.cursor()

    return cursor

# Function to get the list of DBs from the server.
def db_list(server_name):

    cursor = db_connection(server_name)

    #SQL command to get the list of DBs name.
    cursor.execute("select name from sys.databases")
    db_list = []
    rows = cursor.fetchall()
    for row in rows:
        db_list.append(str(row[0]))

    # Return the list from 4th element because the 4 
    # first elements are standard from the SQL Server.
    return db_list[4::]

# Function to get the list of tables name.
def table_list(server_name, db):
    
    cursor = db_connection(server_name, db)
    # SQL command to get the list of tables name.
    cursor.execute('SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES')
    table_list = []
    rows = cursor.fetchall()
    for row in rows:
        table_list.append(str(row[0]))

    # Return list of tables name and server name (cluster format)
    out = (table_list, server_name)   
    return out

def columns_list(server_name, db, table_name):
    cursor = db_connection(server_name, db)
    # SQL Command to select the table.
    cursor.execute("SELECT * FROM {}".format(str(table_name)))
    # Get the list of columns name from the table selected
    columns_list = [column[0] for column in cursor.description]
      
    return columns_list

# Function select data from DB, Table using filter condition.
def filter_table(server_name, db, table ,conditions=['']):
#     conditions is a list of strings, eg. conditions=['Run_ID=513221544']

    cursor = db_connection(server_name, db)
#     print(conditions)
    if len(conditions) == 0:
        # SQL Command to select the table and grab all the data.
        cursor.execute("SELECT * FROM {}".format(str(table)))

    else:
        condition = ""
        for i in range(len(conditions)):
            if i + 1 == len(conditions):
                condition = condition + str(conditions[i])
            else:
                condition = condition + str(conditions[i]) + " And "
        # SQL Command to select the table and filter by condition
        cursor.execute("SELECT * FROM {} WHERE {}".format(str(table),str(condition)))

#         rows is a list of tuples
    rows = cursor.fetchall()
    
    data = []
    for row in rows:        
        sub_data = []
        for x in row:
            x = str(x).replace("\r\n", "")
            sub_data.append(x)
        data.append(sub_data)
    return data

def sql_query(server, db, tbl, sql):
    cursor = db_connection(server_name, db)
    cursor.execute(sql)
    rows = cursor.fetchall()
    data = []
    for row in rows:        
        sub_data = []
        for x in row:
            x = str(x).replace("\r\n", "")
            sub_data.append(x)
        data.append(sub_data)
    return data

def fetch_data(server_name, db, tbl, sql):
    str_connection = 'Driver={SQL Server};Server=' + str(server_name) + '; Trusted_Connection=yes; Database=' + str(db)
    conn = pyodbc.connect(str_connection)
#     sql = 'select * from TX07___ST_Attn_Sweep_vi where Run_ID = 503220929'
    data = pd.read_sql(sql, conn)
    return fetch_data

def convert_to_list(data):
    l = []
    for e in data:
        l.append(e[0])
    return l



In [3]:
# Specify server name
server_name = 'WILMSQL19DEV11'

## Tabulate all the *databases* hosted on the server ##

The table below shows all databases currently hosted by the server 'WILMSQL19DEV11'.

In [4]:
# List all databases in a server
# print("A list of all the databases hosted by the server:")
dbs = db_list(server_name)
df = pd.DataFrame(dbs, columns=['Databases'])
display(df)

,Databases
0,SVB
1,Development
2,Nevis_Dev
3,Nevis_SVB
4,Polaris_Dev
5,Polaris_SVB
6,Navassa_Dev
7,Navassa_SVB
8,RTN_Samples


In [5]:
# Specify database name
db_name = 'Polaris_Dev'

## Tabulate all the *tables* in the database ##

The table below shows all the tables in the selected database.

In [6]:
# List all tables in the database
# print("A list of all tables in the database:")
tables = table_list(server_name, db_name)[0]
df = pd.DataFrame(tables, columns = ['Tables'])
display(df)

,Tables
0,PS_Measurement_PVT2
1,PS_Measurement_PVT
2,RX1_ST_Gain_Sweep_PVT
3,RX7_Intermod_PVT
4,RX1_ST_Gain_Sweep
5,RX3_ST_HD2_Sweep
6,RX4_ST_HD3_Sweep
7,RX5_Self_Spur_Response
8,RX6_Chan_Iso
9,RX7_Intermod


## RX1 ST Gain Sweep

Description of Test

In [7]:
# specify table name
table_name = 'RX1_ST_Gain_Sweep'

Show total number of rows in the table

In [8]:
# Total number of rows in the table
sql = 'SELECT COUNT(id) as count_row from ' + table_name
num_of_rows = sql_query(server_name, db_name, table_name, sql)
print("Total number of rows = ", num_of_rows[0][0])

Total number of rows =  7440


Enumerate all API versions in the table

In [9]:
# Show all API versions available in the table
sql = 'select DISTINCT API_Rev from ' + table_name
api_versions = sql_query(server_name, db_name, table_name, sql)
# convert to dataframe for user-friendly display
df = pd.DataFrame(api_versions, columns = ['API Rev'])
display(df)

,API Rev
0,0


Show an interactive pivot chart with data from the selected table loaded

### Plot 1. RX1 Gain Delta

In [10]:
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)
# plot1 - RX1 Gain Delta Vs Gain Index
pivot_ui(df, rows=['Band','Channel','Set Temp', 'Dut_Id'], cols=['Gain Index'], aggregatorName=['Sum'], hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','Test_Mode','Power_Mode','Use_Case','I GnDelta','Q GnDelta','C GnDelta','I Gain', 'Q Gain', 'C Gain',
        'Noise Figure', 'Noise Power','SNR','NF Delta'],
         rendererName=['Line Chart'], vals=['C GnDelta','Q GnDelta','I GnDelta'], outfile_path='RX1_Gain_Delta_pivot.html', 
          rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})  

In [11]:
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot2 - RX1 Absolute Gain Vs Gain Index
pivot_ui(df, rows=['Band','Channel','Set Temp','Dut_Id'], cols=['Gain Index'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','Test_Mode','Power_Mode','Use_Case','I GnDelta','Q GnDelta','C GnDelta','I Gain', 'Q Gain', 'C Gain',
        'Noise Figure', 'Noise Power','SNR','NF Delta'],
          rendererName=['Line Chart'], vals=['C Gain','Q Gain','I Gain' ], outfile_path='RX1_Absolute_Gain.html',
          rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})


### Plot 3. RX1 NF vs Gidx

In [12]:
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot3 - RX1 Noise Figure Vs Gain Index
pivot_ui(df, rows=['Band','Channel','Set Temp', 'Dut_Id'], cols=['Gain Index'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','Test_Mode','Power_Mode','Use_Case','I GnDelta','Q GnDelta','C GnDelta','I Gain', 'Q Gain', 'C Gain',
        'Noise Figure', 'Noise Power','SNR','NF Delta'],
         rendererName=['Line Chart'],vals=['Noise Figure'], outfile_path='RX1_NF_vs_Gidx.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})    

### Plot 4. RX1 NF vs Gain

In [13]:
# conditions = []
# data = filter_table(server_name, db_name, table_name, conditions)

# cols = columns_list(server_name, db_name, table_name)
# df = pd.DataFrame(data, columns = cols)

# # plot4 - RX1 Noise Figure Vs Gain
# pivot_ui(df, rows=['Band','Channel','Set Temp','Dut_Id'], cols=['C Gain'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
#          'Col3','Col4','Col5','Test_Mode','Power_Mode','Use_Case','I GnDelta','Q GnDelta','C GnDelta','I Gain', 'Q Gain',
#         'Noise Figure', 'Noise Power','SNR','NF Delta'],
#           rendererName=['Line Chart'],vals=['Noise Figure'], outfile_path='RX1_NF_vs_Gain.html',
#           rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})    

### Plot 5. RX1 SNR vs Gidx

In [14]:
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot5 - RX1 SNR Vs Gain Index
pivot_ui(df, rows=['Band','Channel','Set Temp','Dut_Id'], cols=['Gain Index'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','Test_Mode','Power_Mode','Use_Case','I GnDelta','Q GnDelta','C GnDelta','I Gain', 'Q Gain', 'C Gain',
        'Noise Figure', 'Noise Power','SNR','NF Delta'],
         rendererName=['Line Chart'],vals=['SNR'], outfile_path='RX1_SNR_vs_Gidx.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})    

### Plot 2. RX1 Absolute Gain
Click the link below to open the interactive pivot chart.
[RX1 Absolute Gain vs Gidx](files/RX1_Absolute_Gain.html)

## RX7 Intermods IIP3

In [15]:
# specify table name
table_name = 'RX7_Intermod'

Show an interactive pivot chart with data from the selected table loaded

In [16]:
print(table_name)

# fetcg data and store in dataframe df
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot1 - IMD3 High
pivot_ui(df, rows=['Band', 'Channel','Gain Index','Set Temp','Dut_Id'], cols=['Input Pwr'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','CH1 Band Center','CH2 Band Center','Use_Case','CH3 Band Center ','CH4 Band Center','SG1 Offset dB','SG2 Offset dB', 'Pin', 'IMD3 High',
        'IMD3 Low','IIP3 Low','IIP3 High','F1 Pwr ','F2 Pwr','F1 Hz','F2 Hz','2F1-F2 Pwr','2F2-F1 Pwr','2F1-F2 Hz','2F2-F1 Hz','Power_Mode','Test_Mode', 'Gain Index'],
         rendererName=['Line Chart'], vals=['IMD3 High'], outfile_path='RX7_IMD3_High.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})        

RX7_Intermod


In [17]:
print(table_name)

# fetcg data and store in dataframe df
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot2 - IMD3 Low
pivot_ui(df, rows=['Band', 'Channel','Gain Index','Set Temp','Dut_Id'], cols=['Input Pwr'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','CH1 Band Center','CH2 Band Center','Use_Case','CH3 Band Center ','CH4 Band Center','SG1 Offset dB','SG2 Offset dB', 'Pin', 'IMD3 High',
        'IMD3 Low','IIP3 Low','IIP3 High','F1 Pwr ','F2 Pwr','F1 Hz','F2 Hz','2F1-F2 Pwr','2F2-F1 Pwr','2F1-F2 Hz','2F2-F1 Hz','Power_Mode','Test_Mode', 'Gain Index'],
         rendererName=['Line Chart'], vals=['IMD3 Low'], outfile_path='RX7_IMD3_Low.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})

RX7_Intermod


In [18]:
print(table_name)

# fetcg data and store in dataframe df
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot3 - IIP3 High
pivot_ui(df, rows=['Band', 'Channel','Gain Index','Set Temp','Dut_Id'], cols=['Input Pwr'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','CH1 Band Center','CH2 Band Center','Use_Case','CH3 Band Center ','CH4 Band Center','SG1 Offset dB','SG2 Offset dB', 'Pin', 'IMD3 High',
        'IMD3 Low','IIP3 Low','IIP3 High','F1 Pwr ','F2 Pwr','F1 Hz','F2 Hz','2F1-F2 Pwr','2F2-F1 Pwr','2F1-F2 Hz','2F2-F1 Hz','Power_Mode','Test_Mode', 'Gain Index'],
         rendererName=['Line Chart'], vals=['IIP3 High'], outfile_path='RX7_IIP3_High.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})

RX7_Intermod


In [19]:
print(table_name)

# fetcg data and store in dataframe df
conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)

# plot3 - IIP3 Low
pivot_ui(df, rows=['Band', 'Channel','Gain Index','Set Temp','Dut_Id'], cols=['Input Pwr'], aggregatorName=['Sum'],hiddenFromDragDrop=['Col1','Col2',
         'Col3','Col4','Col5','CH1 Band Center','CH2 Band Center','Use_Case','CH3 Band Center ','CH4 Band Center','SG1 Offset dB','SG2 Offset dB', 'Pin', 'IMD3 High',
        'IMD3 Low','IIP3 Low','IIP3 High','F1 Pwr ','F2 Pwr','F1 Hz','F2 Hz','2F1-F2 Pwr','2F2-F1 Pwr','2F1-F2 Hz','2F2-F1 Hz','Power_Mode','Test_Mode', 'Gain Index'],
         rendererName=['Line Chart'], vals=['IIP3 Low'], outfile_path='RX7_IIP3_Low.html',
         rendererOptions = {"c3": {"size":{"width":600, "height":300}, "legend":{"hide": True}}})

RX7_Intermod


### Plot 1. IIMD3 High vs Input Pwr
Click the link below to open the interactive pivot chart.
[RX7 IMD3 High](files/RX7_IMD3_High.html)
### Plot 2. IIMD3 Low vs Input Pwr
Click the link below to open the interactive pivot chart.
[RX7 IMD3 Low](files/RX7_IMD3_Low.html)
### Plot 3. IIP3 High vs Input Pwr
Click the link below to open the interactive pivot chart.
[RX7 IIP3 High](files/RX7_IIP3_High.html)
### Plot 4. IIP3 Low vs Input Pwr
Click the link below to open the interactive pivot chart.
[RX7 IIP3 Low](files/RX7_IIP3_Low.html)

## RX3 ST HD3 Sweep

Add Description of Test

In [20]:
# specify table name
table_name = 'RX4_ST_HD3_Sweep'

Show an interactive pivot chart with data from the selected table loaded

In [21]:

conditions = []
data = filter_table(server_name, db_name, table_name, conditions)

cols = columns_list(server_name, db_name, table_name)
df = pd.DataFrame(data, columns = cols)
# plot1 - IMD3 High
pivot_ui(df, rows=['Band', 'HD3 Fout','Set Temp'], cols=['Channel'], aggregatorName=['Sum'],
         rendererName=['Bar Chart'], vals=['HD3 dBc'], outfile_path='RX4_HD3.html',
         rendererOptions = {"c3": {"legend": {"hide": ['Band', 'HD3 Fout','Set Temp']}}})
    

### Plot 1.HD3 Pout dBc
Click the link below to open the interactive pivot chart.
[RX3 HD3](files/RX4_HD3.html)